# 逻辑回归（Logistic Regression）

逻辑回归是一种用于二分类问题的统计学习方法，常用于预测事件的发生概率（例如成功/失败、是/否）。

### 与线性回归的区别
线性回归：拟合一条直线以预测连续值，关注变量之间的线性关系。

逻辑回归：使用非线性函数（如 Sigmoid）预测事件发生的概率，输出值在 0 到 1 之间。

### 使用 Sigmoid 函数
逻辑回归的核心是 Sigmoid 函数，它将任意实数映射到 (0, 1) 区间：

$$
\sigma(z) = \frac{1}{1 + e^{-z}}
$$

其中：
$$
z = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \dots + \beta_n x_n
$$

$𝛽_0$：截距项（bias）

$𝛽_𝑖$：第$𝑖$个特征的权重系数

$𝑥_𝑖$：第$𝑖$个输入特征值

## Sigmoid 输出解释

| 𝑧 值范围 | Sigmoid 输出 $$\sigma(z)$$ | 预测含义                     |
|-----------------|------------------------|--------------------------|
| $$( z \ll 0 )$$   | 接近 0               | 否定性预测（事件不发生） |
| $$( z \gg 0 )$$  | 接近 1               | 肯定性预测（事件发生）   |
| $$( z \approx 0 )$$ | 接近 0.5          | 模糊边界（不确定）       |


In [1]:
import numpy as np

# Sigmoid 函数定义
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# 线性组合函数：z = β0 + β1*x1 + β2*x2 + ...
def linear_combination(X, weights, bias):
    return np.dot(X, weights) + bias

# 预测函数：返回概率
def predict_proba(X, weights, bias):
    z = linear_combination(X, weights, bias)
    return sigmoid(z)

# 二分类预测：返回 0 或 1
def predict(X, weights, bias, threshold=0.5):
    proba = predict_proba(X, weights, bias)
    return (proba >= threshold).astype(int)

# 示例数据
X_sample = np.array([2.0, -1.0])        # 特征值
weights = np.array([0.8, -0.5])         # 权重
bias = 0.2                              # 截距

# 执行预测
probability = predict_proba(X_sample, weights, bias)
label = predict(X_sample, weights, bias)

print(f"预测概率: {probability:.4f}")
print(f"预测类别: {label}")


预测概率: 0.9089
预测类别: 1


## 最大似然估计法（Maximum Likelihood Estimation）

在逻辑回归中，我们希望找到一组参数，使得模型对训练数据的预测概率最大。为此，我们定义一个**似然函数**，并通过优化使其最大化。

### 数学表达

假设训练数据为 $(x_i, y_i)$，其中 $ y_i \in \{0, 1\} $，则逻辑回归的似然函数为：

$$
L(\beta) = \prod_{i=1}^{n} \sigma(z_i)^{y_i} (1 - \sigma(z_i))^{1 - y_i}
$$

我们通常对数化处理，得到对数似然函数：

$$
\log L(\beta) = \sum_{i=1}^{n} \left[ y_i \log \sigma(z_i) + (1 - y_i) \log (1 - \sigma(z_i)) \right]
$$

然后使用梯度下降法等优化算法，求出使对数似然函数最大化的参数𝛽。

In [2]:
import numpy as np

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def log_likelihood(X, y, beta):
    z = X @ beta
    return np.sum(y * np.log(sigmoid(z)) + (1 - y) * np.log(1 - sigmoid(z)))

def logistic_regression(X, y, lr=0.01, max_iter=10000, tol=1e-6):
    m, n = X.shape
    beta = np.zeros(n)
    prev_ll = -np.inf

    for i in range(max_iter):
        z = X @ beta
        gradient = X.T @ (y - sigmoid(z))
        beta += lr * gradient

        ll = log_likelihood(X, y, beta)
        if np.abs(ll - prev_ll) < tol:
            break
        prev_ll = ll

    return beta

# 示例数据
X = np.array([
    [1, 0.5],
    [1, 1.0],
    [1, 1.5],
    [1, 2.0],
    [1, 2.5],
    [1, 3.0],
    [1, 3.5],
    [1, 4.0],
    [1, 4.5],
    [1, 5.0]
])  # 第一列是偏置项

y = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])

beta_hat = logistic_regression(X, y)
print("估计的参数 β:", beta_hat)


估计的参数 β: [-13.50758443   4.9570286 ]


### 说明

- 使用的是梯度上升法（最大化对数似然）

- X 的第一列是常数项（偏置项）

- beta_hat 是估计得到的参数向量，可用于预测新样本的概率

下面是一个使用 scikit-learn 实现逻辑回归的完整 Python 示例，适用于二分类问题，并包含训练、预测和评估步骤。

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 示例数据：你可以替换为自己的 CSV 文件
data = pd.DataFrame({
    'feature1': [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0],
    'feature2': [1, 2, 1, 2, 1, 2, 1, 2, 1, 2],
    'target':   [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
})

# 特征和目标变量
X = data[['feature1', 'feature2']]
y = data['target']

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 初始化并训练模型
model = LogisticRegression()
model.fit(X_train, y_train)

# 预测
y_pred = model.predict(X_test)

# 评估
print("准确率 Accuracy:", accuracy_score(y_test, y_pred))
print("混淆矩阵 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("分类报告 Classification Report:\n", classification_report(y_test, y_pred))

# 可选：预测概率
y_prob = model.predict_proba(X_test)
print("预测概率 Predicted Probabilities:\n", y_prob)


准确率 Accuracy: 0.6666666666666666
混淆矩阵 Confusion Matrix:
 [[1 0]
 [1 1]]
分类报告 Classification Report:
               precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       1.00      0.50      0.67         2

    accuracy                           0.67         3
   macro avg       0.75      0.75      0.67         3
weighted avg       0.83      0.67      0.67         3

预测概率 Predicted Probabilities:
 [[0.15205575 0.84794425]
 [0.92529562 0.07470438]
 [0.51048435 0.48951565]]


### 说明

- LogisticRegression() 默认使用 L2 正则化和 lbfgs 求解器

- predict() 给出类别预测（0 或 1）

- predict_proba() 给出属于每个类别的概率

- 可通过 penalty, solver, C, max_iter 等参数进行高级配置